In [ ]:
!pip install bertopic

In [ ]:
# training 19 BERTopic models
import pandas as pd
import numpy as np
from bertopic import BERTopic

df = pd.read_csv("./osr_tweets_without_S_T_U_U_v2.csv", engine='python')
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/osr_tweets_without_T_U_U_v2.csv", engine='python')


def get_num_unique_topics(merged_topic):
    exist_topic = []
    for topic in merged_topic:
        if not topic in exist_topic:
            exist_topic.append(topic)
    print(len(exist_topic))

df.text.astype(str)
df.dropna()
df = df[df['text'].apply(lambda x:not isinstance(x, float)  and len(x)>=10)]
df = df.reset_index(drop=True)

num_topics_list = range(10,101,5)

topic_cluster_list = []
for num_topics in num_topics_list:

    # create model
    model = BERTopic(verbose=True, nr_topics=num_topics)

    #convert to list
    docs = df.text.to_list()
    topics, probabilities = model.fit_transform(docs)

    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer_model = CountVectorizer(stop_words="english")
    model.update_topics(docs, vectorizer_model=vectorizer_model)

    # make predicts for all the tweets in the docs
    predicted, probs = model.transform(docs)
    pred_list = predicted

    # exclude the topic for clustet -1, as it is the cluster for outlier tweets
    topic_for_each_cluster = []
    for i in range(0,num_topics-1):
        cur_index_list = []
        topic2counter = {}
        # reserve all the index predicted as ith cluster
        for j in range(len(pred_list)):
            if pred_list[j] == i:
                cur_index_list.append(j)
        # from all the tweet in ith cluster, count their appearance time
        for idx in cur_index_list:
            topic = df.topic[idx]
            if topic in topic2counter:
                topic2counter[topic] += 1
            else:
                topic2counter[topic] = 1

        topic4i = ""
        num4i = 0
        # assign topic for current cluster to be the most appeared topic
        for topic, counter in topic2counter.items():
            if counter > num4i:
                topic4i = topic

        topic_for_each_cluster.append(topic4i)

    print(topic_for_each_cluster)
    get_num_unique_topics(topic_for_each_cluster)
    topic_cluster_list.append(topic_for_each_cluster)
    filename = '{num_topics}_bertopic_model'.format(num_topics = num_topics)
    model.save(filename)

In [7]:
# generate label2id map

id2label = {
0 : 'Children Education and Skills',
1 : 'Health and Social Care',
2 : 'Crime and Security',
3 : 'Economy',
4 : 'Housing Planning and Local Services',
5 : 'Labour Market and Welfare',
6 : 'Population and Society',
7 : 'Transport Environment and Climate Change'
}
map = {}
for id, label in id2label.items():
    map[label] = id

In [ ]:
# evaluate 19 topic modeling models on test set

import pandas as pd

df = pd.read_csv("./test_set.csv", engine='python')
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/test_set.csv", engine='python')

for num_topics, topic_cluster in zip(num_topics_list, topic_cluster_list):
    filename = '{num_topics}_bertopic_model'.format(num_topics = num_topics)
    model = BERTopic.load(filename)
    docs = df.text.to_list()
    predicted, probs = model.transform(docs)
    predicted_classes = []
    for cluster_num in predicted:
        if cluster_num == -1:
            predicted_classes.append(-1)
        else:
            predicted_classes.append(map[topic_cluster[cluster_num]])
    #  [topic_cluster[cluster_num] for cluster_num in predicted if not cluster == -1 else -1]
    accuracy = np.mean(np.array(predicted_classes) == df.label.values)
    print('Number of topic:', num_topics, 'Accuracy:',accuracy)